In [ ]:
# first mount your google drive
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CSC420/Project"

Mounted at /content/drive
img.png  mask.png  Priority.ipynb


In [38]:
# the usual
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import feature

In [42]:
# Get source region and target region and init confidence
def load_image(image1, image2):
    image = cv2.imread(image1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = cv2.imread(image2)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    confidence = (mask == 0).astype('float')
    return image, mask, confidence

In [43]:
image, mask, confidence = load_image("/content/drive/My Drive/CSC420/Project/img.png","/content/drive/My Drive/CSC420/Project/mask.png")

In [124]:
# Use edge detection to get front fill Delta Omega from mask image
def get_front_fill(mask):
  ff = cv2.Scharr(mask, cv2.CV_64F)
  itemindex = np.where(ff==1)
  return itemindex

In [159]:
# Use point axis and window_size to get the range of patch psi P(2X2 array)
def patch(image,point, window_size):
  iy,ix = image.shape
  x, y = point
  size = window_size[0]//2
  x_1 = max(x - size, 0)
  y_1 = max(y - size, 0)
  x_2 = min(x + size, ix - 1)
  y_2 = min(y + size, iy - 1)
  return [(x_1, y_1),(x_2, y_2)]

In [264]:
# compute point p confidence
def compute_confidence(confidence, point, mask, window_size):
  psiP = patch(mask, point, window_size)
  x_1, y_1 = psiP[0]
  x_2, y_2 = psiP[1]
  area = (x_2-x_1) * (y_2-y_1)
  t_sum = 0
  for i in range(x_1, x_2 + 1):
      for j in range(y_1, y_2 + 1):
          if mask[j, i] == 0:
              t_sum += confidence[j, i]
  confidence[point[1],point[0]] = t_sum / area
  return confidence,t_sum / area

In [243]:
# Get the gradient of isophote around the patch
def compute_gradient(image,mask):
  alpha = 255
  gx = (cv2.Scharr(image, cv2.CV_64F, 1, 0))/alpha
  gy = (cv2.Scharr(image, cv2.CV_64F, 0, 1))/alpha
  gx[mask==1]=0
  gy[mask==1]=0
  return gx,gy

In [211]:
# Get the unit vector of the normal on front fill
def compute_normal(front_fill,mask):
  gx = cv2.Scharr(mask, cv2.CV_64F, 1, 0)
  gy = cv2.Scharr(mask, cv2.CV_64F, 0, 1)
  normal = []
  for i in range(len(front_fill[0])):
    x,y = front_fill[1][i], front_fill[0][i]
    norm = np.sqrt(gx[y,x]**2 + gy[y,x]**2)
    if norm != 0:
      normal.append([gy[y,x]/norm,-gx[y,x]/norm])
    else:
      normal.append([gy[y,x],-gx[y,x]])
  return normal


In [239]:
# Compute data term by equation
def compute_data(nalba_Ip, normal, point, point_num):
  x,y = point
  alpha = 255
  data = (np.sqrt((nalba_Ip[0][y,x] * normal[point_num][0])**2 + (nalba_Ip[1][y,x] * normal[point_num][1])**2)) / alpha
  return data

In [231]:
# Compute priority for point P
def compute_priority(confidence, data):
  priority = confidence * data
  return priority


In [265]:
image, mask, confidence = load_image("/content/drive/My Drive/CSC420/Project/img.png","/content/drive/My Drive/CSC420/Project/mask.png")
window_size = (3,3)
front_fill = get_front_fill(mask)
nalba_Ip = compute_gradient(image,mask)
normal = compute_normal(front_fill,mask)
ff_priority = []
for i in range(len(front_fill[0])):
  curr = (front_fill[1][i],front_fill[0][i])
  confidence, curr_confidence = compute_confidence(confidence,curr,mask,window_size)
  curr_data = compute_data(nalba_Ip,normal,curr,i)
  ff_priority.append(compute_priority(curr_confidence,curr_data))
curr_max = np.argmax(ff_priority)
curr_idx = (front_fill[1][curr_max],front_fill[0][curr_max])

In [ ]:
#Update confidence in intersection aera of sai P_hat and omega with C(P_hat)
def update_confidence(confidence, saiP_hat, P_hat):
  px,py = P_hat
  curr = confidence[py,px]
  x_1, y_1 = saiP_hat[0]
  x_2, y_2 = saiP_hat[1]
  for i in range(x_1, x_2 + 1):
      for j in range(y_1, y_2 + 1):
        if confidence[j,i] ==0:
          confidence[j,i] = curr
    return confidence